In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from tqdm import notebook

In [3]:
def sectorspread():
    
    zcb = pd.read_csv('bootstrap_ZCB_ytm.csv', index_col = 'date')
    zcb.index = pd.to_datetime(zcb.index.astype(str), format = '%Y-%m-%d')
    zcb = 100*zcb
    
    sector_dat = pd.read_csv('sector_data.csv', index_col='date')
    sector_dat.index = pd.to_datetime(sector_dat.index)
    
    '''sector_result = sector_dat[(sector_dat['industry_m'] == '금융업')] ##| (sector_dat['industry_m'] == '섹터이름2')] ## |연산으로 추가
    sector_result.index = pd.to_datetime(sector_result.index.astype(str))'''
    
    sector_result = sector_dat
    
    table = pd.pivot_table(sector_result, values = ['duration', '만기수익률'], index = ['code', 'date'])
    table.drop(index = table[table['duration'].isna() == True].index, inplace = True)
    table.drop(index = table[table['만기수익률'].isna() == True].index, inplace = True)
    
    mat = []
    tn = []
    
    for i in notebook.tqdm(range(len(table.index))):
        
        duration = table["duration"].loc[table.index[i]]
        ytm = table['만기수익률'].loc[table.index[i]]
        
        if float(duration) <= 0:
            duration = 3
        elif float(duration) <= 1:
            duration = round(duration/0.25)*3 #분위수 개념 ()
        elif float(duration) <= 3:
            duration = round(duration/0.5)*6  #분위수 개념 ()
        elif round(float(duration)) == 6:
            duration = 60 #duration 이 반올림하여 6년이면, 5년 zcb와 matching
        elif round(float(duration)) == 8:
            duration = 84 #duration 이 반올림하여 8년이면, 7년 zcb와 matching
        elif round(float(duration)) == 9:
            duration = 120 #duration 이 반올림하여 9년이면, 10년 zcb와 matching  
        elif round(float(duration)) > 10:
            duration = 120   #duration 이 반올림하여 10년보다 크면, 10년 zcb와 matching
        else:
            duration = round(float(duration))*12 #나머진 전부 개월로 변환
        
        mat.append(duration)
        
        try:
            tn.append(zcb.loc[table.index[i][-1]]['MATURITY'+str(duration)])
        
        except Exception:
            tn.append(np.nan)
            
    table['국고채만기'] = mat
    table['spread'] = table['만기수익률']-tn
    table['spread'] = 100*table['spread']
        
    return table

In [4]:
sectorwide = sectorspread()

In [5]:
sectorwide

duration     만기수익률  국고채만기    spread
code    date                                           
A000010 2010-01-04  3.678266  5.006566     48  28.43862
        2010-01-05  3.657210  4.931621     48  28.84949
        2010-01-06  3.639803  4.942637     48  24.46884
        2010-01-07  3.663052  4.867548     48  27.47671
        2010-01-08  3.673980  4.859864     48  21.44042
...                      ...       ...    ...       ...
A316140 2020-08-25  3.975537  2.867630     48       NaN
        2020-08-26  3.974757  2.874615     48       NaN
        2020-08-27  3.976660  2.882189     48       NaN
        2020-08-28  3.982847  2.904151     48       NaN
        2020-08-31  3.982671  2.922710     48       NaN

[709872 rows x 4 columns]